In [17]:
import numpy as np
import pandas as pd
import os
from itertools import combinations, combinations_with_replacement, product
import wntr
import random
import networkx as nx
import copy

In [18]:
# Getting path for the 'parent folder'
path_cwd = os.getcwd()
path_parent = os.path.abspath(os.path.join(path_cwd, os.pardir))

In [19]:
# Getting path for the input file
inputfiles_folder_name = 'Input_files_EPANET'
filename = 'Hanoi_leak_14.inp'
path_file = os.path.join(path_parent,inputfiles_folder_name,filename)

In [20]:
# Reading the input file into EPANET
inp_file = path_file
wn = wntr.network.WaterNetworkModel(inp_file)
wn1 = wntr.network.WaterNetworkModel(inp_file)

In [21]:
# Getting path for the 'No leak datafile' that contains expected states of given nodal demands and resulting flows and pressures
# across the WDN
data_folder_name = 'Data_files'
data_filename = 'data_base_demand_leakgen.csv'
path_data_file = os.path.join(path_parent,data_folder_name,data_filename)

## Creating empty dataframes with suitably named columns, to be used to store the outputs with leak

In [22]:
# Here we run an EPANET simulation only to create sample output from a simulation
# and store the column headers of those results. These headers will be used in a file to store results from the 
# "leak experiment" 

sim_leak = wntr.sim.EpanetSimulator(wn1)
results = sim_leak.run_sim()
df_demand = results.node['demand']
df_head = results.node['head']
df_flow = results.link['flowrate']

In [38]:
# Use the column names from the results in cell above and create empty dataframes to store results of 
# leak experiment 
demand_data = pd.DataFrame(columns = df_demand.columns)
leak_demand_data = pd.DataFrame(columns = df_demand.columns)
head_data = pd.DataFrame(columns = df_demand.columns)
flow_data = pd.DataFrame(columns = df_flow.columns)

# 'Expected states' stand for 'No Leak' demand cases that are being used for leak simulation.
# The underlying demand, resulting flow and pressure without any leak are stored along with the 
# resulting flow and pressure with leak.

expected_states = pd.read_csv(path_data_file)
expected_states = expected_states.sample(frac=0.25)
expected_states = expected_states.reset_index(drop=True)
expected_state_data = pd.DataFrame(columns=expected_states.columns)
expected_states_np = np.array(expected_states)

In [39]:
# initializing some key values
num_nodes = 32
num_links = 34
train_samples = expected_states_np.shape[0] # No of demand scenarios to be fed to EPANET with leak in place
num_total_leaks = 1 # For current set of experiments

In [40]:
# create list of names of all nodes and links
link_name = wn1.link_name_list
node_name = wn1.node_name_list
leak_node_name = node_name[num_nodes-1:-1] # The leak node is assigned number one greater than the last normal node
leaking_node = pd.DataFrame(columns = leak_node_name)

In [41]:
#Creating more empty dataframes to store results: one for storing the demand scenario being fed into EPANET during
# the experiment, another to store the area of the leak(s) that are active during the experiment
demand_data_in = pd.DataFrame(columns = df_demand.columns[:-1])
area_in = pd.DataFrame(columns=leak_node_name)

## Next two functions are used to for creating leak combinations in case of multiple leaks and for assigning 'area' to the leaks, which is assumed to be a circular orifice. Note that multiple leaks at a time along with multiple possible leak sizes would lead to a large number of possible combinations. In the current study, we have assumed only one leak at a time 

In [42]:
# below code creates a list of nCp combinations for 'n' links with leak nodes and 'p' total leaks in the system. 
# Here we have assumed that EACH PIPE CAN HAVE ONLY ONE LEAK AT A TIME. Since we have only one leak, 
# its a single element list.
def leak_combs(num_tot_leaks):
         
    combs = list(combinations(leak_node_name,num_tot_leaks))
    
    return combs                 

In [43]:
# For a given combination of leak nodes, say 2 leak nodes, each node may have a range of leak areas. The code
# below first creates a list of areas and then creates nPc combinations where 'n' is the no of areas and 'c' 
# is no of leaks
def leak_area_combs(area_list):    
    combs = list(combinations_with_replacement(area_list,num_total_leaks))    
    return combs

In [44]:
# A hole of radius 2 cm would have an area of around 1e-3 m2 and hole with radius 12 cm would have 
# around 50e-3 m2. We are assuming a leak in this area range. Default emmitter coefficient is 0.75 

area_list = leak_area_combs([0.0001, 0.0005, 0.001 , 0.002 , 0.003 , 0.004 , 0.005 ])            

In [45]:
area_list

[(0.0001,), (0.0005,), (0.001,), (0.002,), (0.003,), (0.004,), (0.005,)]

## The loop below takes in the demand values from each of the 'No Leak datafile' ('expected states in the code below'). These demand values are then fed into the EPANET along with a leak. However, for a given demand scenario, multiple leak sizes are used. The resulting flow and head values are stored along with the leak size and leak demand

In [46]:
k=0
for i in range(train_samples):
    wn = wntr.network.WaterNetworkModel(inp_file)
    # set up the reservoir head as per 'No leak datafile'
    wn.get_node(1).head_timeseries.base_value = expected_states_np[i,63]
    
    # set up demands across junction nodes as per 'No leak datafile'
    
    for n in range(2,num_nodes+1):            
        wn.get_node(n).demand_timeseries_list[0].base_value = expected_states_np[i,n-2]       
  
        
    # Loop 1: over each of the leakage node combinations in the network
    for leak_nodes in leak_combs(num_total_leaks):           

        # Loop 3: over each of the Emitter coefficient value combination (its a fixed list given no of leaks)
        for a in range(len(area_list)):
                        
            wn_sim = copy.deepcopy(wn)
            leaking_node_data = [] # to store leak node names and corresponding ECs

            # Loop 4: over each of the leakage node for a leakage node combination and EC combination
            for b in range(num_total_leaks):
                wn_sim.get_node(leak_nodes[b]).add_leak(wn_sim,area = area_list[a][b], start_time=0)                
                leaking_node_data.append(leak_nodes[b])
                leaking_node_data.append(area_list[a][b])                     

            l_leak = []
            for leaknode in leak_node_name:
                l_leak.append(wn_sim.get_node(leaknode).leak_area)
            area_in.loc[k] = l_leak                

            sim = wntr.sim.WNTRSimulator(wn_sim)
            results = sim.run_sim()
            demand_data.loc[k] = results.node['demand'].loc[0]
            leak_demand_data.loc[k] = results.node['leak_demand'].loc[0]
            head_data.loc[k] = results.node['head'].loc[0]
            flow_data.loc[k] = results.link['flowrate'].loc[0]
            expected_state_data.loc[k] = expected_states.loc[i]                     

            k=k+1
            

## The code below stores the data generated by simulator in a single dataframe. Naming of the columns is important and need to be taken care of

In [47]:
leak_demand = demand_data.copy()
leak_demand_leak = leak_demand_data.copy()
leak_flow = flow_data.copy()
leak_head = head_data.copy()
leak_area = area_in.copy()
leak_expected = expected_state_data.copy()

In [48]:
leakheadnames = []
for x in list(leak_head):
    y = 'leak_head_'+x
    leakheadnames.append(y)
    
leak_head.columns = leakheadnames

leakflownames = []
for x in list(leak_flow.columns):
    y = 'leak_flow_'+x
    leakflownames.append(y)
    
leak_flow.columns=leakflownames

leak_area.columns=['leak_area']

leakage_demand = pd.DataFrame(columns=['leakage_demand'])
leakage_demand['leakage_demand'] = leak_demand_leak['33']

leak_combined = pd.concat((leak_area,leakage_demand,leak_head,leak_flow,leak_expected),axis=1)


## Saving the dataframe as csv in the Data_files folder

In [49]:
# Output folder name defined
datafiles_folder_name = 'Data_files'

# Output file names defined
leakfile = 'data_leak_in_14.csv'

# Creating file paths. Note that 'path_parent' has been defined earlier
path_leak_file = os.path.join(path_parent,datafiles_folder_name,leakfile)

# Creating the 'Data_files' folder, if it doesn't already exists
os.makedirs(os.path.dirname(path_leak_file), exist_ok=True)

# Saving the output datasets as csv files whose paths have been defined above
leak_combined.to_csv(path_leak_file,index=None)

In [50]:
#check shape
leak_combined.shape

(17500, 168)